In [1]:
from pathlib import Path

base_path = Path.cwd()

# Lendo os dados

Lembre-se de ter rodado o notebook anterior antes de ler os dados aqui!

In [2]:
from projeto.dataset import load_processed_datasets

X, y, X_train, X_test, y_train, y_test = load_processed_datasets(base_path)

# Escolha de modelo

Teste os seguintes modelos:

- Dummy classifier

- Logistic regression com regularização L2 (`sklearn.linear_model.LogisticRegression(penalty='l2', C=1.0)`) para valores de C entre $10^{-3}$ e $10^{3}$

- Logistic regression com regularização L2 como no caso anterior e features polinomiais. Experimente com grau polinomial entre $2$ e $3$

- Classificador do tipo Support Vector Machines com kernel polinomial (`sklearn.svm.SVC(C=1.0, degree=2)`), teste valores de C entre $10^{-3}$ e $10^{3}$ e graus entre $1$ e $5$.

    - Atenção: este modelo demora MUITO para treinar, portanto vamos treiná-lo com um subconjunto PEQUENO de amostras. Você pode usar o seguinte código para construir um *dataset* de treino pequeno:

        ```Python
        subset_indexes = np.random.choice(X_train.index, size=5000, replace=False, random_state=42)
        X_train_subset = X_train.loc[subset_indexes]
        y_train_subset = y_train.loc[subset_indexes]
        ```

    Somente o `SVC` será treinado com um *dataset* pequeno.

- Classificador do tipo Random Forest (`sklearn.ensemble.RandomForestClassifier(n_estimators=100, max_depth=10, max_features=1.0, n_jobs=-1)`), varie o parâmetro `n_estimators` com os valores $\{2, 8, 32, 128, 512\}$, o parâmetro `max_depth` variando em $\{1, 4, 16, 64, \text{None}\}$ e o parâmetro `max_features` variando em $\{0.1, 0.3, 0.5, 0.7, 1.0\}$

- Classificador do tipo `sklearn.ensemble.HistGradientBoostingClassifier(max_depth=None, max_features=1.0)` com o parâmetro `max_depth` variando em $\{1, 4, 16, 64, \text{None}\}$ e o parâmetro `max_features` variando em $\{0.1, 0.3, 0.5, 0.7, 1.0\}$

Construa pipelines adequadas, e use o `sklearn.model_selection.GridSearchCV` para escolher o melhor modelo de cada categoria, e então determine qual o melhor modelo de todos.


# Certificação

Para o modelo escolhido:

- Efetue a etapa de certificação.

- Construa a curva ROC deste modelo e calcule a área sobre a curva ROC.

- Discuta os resultados.

# Deploy

Retreine o modelo no conjunto completo de dados.

Demonstre como você:

- Grava o modelo treinado no disco

- Lê o modelo treinado do disco

- Usa o modelo treinado para fazer uma predição para um novo item.